In [91]:
import index_database as db
import abc

In [92]:
class FulltextSearchEngineBase(abc.ABC):
    @abc.abstractmethod
    def get_ranked_pages(query : list[str]) -> dict[db.Urls, float]:
        ...

In [93]:
class DocumentAtATimeSearchEngine(FulltextSearchEngineBase):

    def get_ranked_pages(query : list[str]) -> dict[db.Urls, int]:
        all_pages = db.Urls.get_all_indexed_pages()
        result = {}

        for page in all_pages:
            result[page] = DocumentAtATimeSearchEngine.calculate_score(page, query)

        return result
    
    def calculate_score(page : db.Urls, query : list[str]) -> int:
        score = 0
        for query_entry in query:
            word_object = db.Words.get_or_none(db.Words.term == query_entry)
            if(word_object is None):
                continue

            word_reference_object = db.WordsReferences.get_or_none(db.WordsReferences.referenced_word == word_object, 
                                           db.WordsReferences.page_url == page)
            if(word_reference_object is None):
                continue

            score += word_reference_object.count

        return score
        

In [94]:
class TermAtATimeSearchEngine(FulltextSearchEngineBase):

    def get_ranked_pages(query : list[str]) -> dict[db.Urls, int]:
        all_pages = db.Urls.get_all_indexed_pages()
        result = {}

        for page in all_pages:
            result[page] = 0

        for term in query:
            for page in all_pages:
                result[page] += TermAtATimeSearchEngine.calculate_score(page, term)
        
        return result
    
    def calculate_score(page : db.Urls, term : str) -> int:
        word_object = db.Words.get_or_none(db.Words.term == term)
        if(word_object is None):
            return 0
        
        word_reference_object = db.WordsReferences.get_or_none(db.WordsReferences.referenced_word == word_object, 
                                           db.WordsReferences.page_url == page)
        if(word_reference_object is None):
            return 0 
        
        return word_reference_object.count

In [95]:
def handle_user_input(input : str, search_engine : FulltextSearchEngineBase):
    query = input.lower().split()
    return search_engine.get_ranked_pages(query)

In [96]:
user_input = input("Your query")

In [97]:
result = handle_user_input(user_input, DocumentAtATimeSearchEngine)

sorted_result = sorted(result, key = lambda key : result[key], reverse = True)

for page in sorted_result:
    print('Url:\t', page.url)
    print('Score:\t', result[page])
    print()

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html
Score:	 229

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/DownstreamDev.html
Score:	 131

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/Compatibility.html
Score:	 126

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SecureMode.html
Score:	 122

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/ClusterSetup.html
Score:	 118

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/AdminCompatibilityGuide.html
Score:	 101

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CommandsManual.html
Score:	 74

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
Score:	 61

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/NativeLibraries.html
Score:	 46

Ur

In [98]:
result = handle_user_input(user_input, TermAtATimeSearchEngine)

sorted_result = sorted(result, key = lambda key : result[key], reverse = True)

for page in sorted_result:
    print('Url:\t', page.url)
    print('Score:\t', result[page])
    print()

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html
Score:	 229

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/DownstreamDev.html
Score:	 131

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/Compatibility.html
Score:	 126

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SecureMode.html
Score:	 122

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/ClusterSetup.html
Score:	 118

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/AdminCompatibilityGuide.html
Score:	 101

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CommandsManual.html
Score:	 74

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
Score:	 61

Url:	 https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/NativeLibraries.html
Score:	 46

Ur